In [1]:
import configparser

import requests
# from parsel import Selector

import pandas as pd

In [2]:
subreddit      = "SanDiego"
subreddit_filt = "new"

In [3]:
# read in config file
config = configparser.ConfigParser()
config.read('reddit_config.ini')

['reddit_config.ini']

In [4]:
# Request access to reddit
auth = requests.auth.HTTPBasicAuth(config["script"]["personal_use_script"], config["script"]["secret"])

# set up log in creds
data = {'grant_type': 'password',
        'username': config["auth"]["username"],
        'password': config["auth"]["password"]
       }

headers = {'User-Agent': 'test_project'}

In [5]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [6]:
# res = requests.get("https://oauth.reddit.com/r/SanDiego/best",
#                    headers=headers, params={'limit':'5'})
res = requests.get(f"https://oauth.reddit.com/r/{subreddit}/{subreddit_filt}",
                   headers=headers, params={'limit':'200'})

In [7]:
reddit_res = res.json()

In [8]:
keys_interest = ['title', 'created_utc', 'selftext', 'url', 'link_flair_text']
data_dict = {}
for i, d in enumerate(reddit_res['data']['children']):
    data_dict[i] = {k: d['data'][k] for k in keys_interest}
data_df = pd.DataFrame(data_dict).T
data_df

,title,created_utc,selftext,url,link_flair_text
0,4240 La Jolla Village Marriott is swarming wit...,1667534410.0,,https://i.redd.it/onsyzioxfwx91.jpg,Photo
1,Air Force One with POTUS Landing at MCAS Miram...,1667531973.0,,https://v.redd.it/aehmjo73rux91,Video
2,How bad are layoffs at your company?,1667528580.0,,https://www.reddit.com/r/sandiego/comments/ylm...,None
3,Any photos or videos of Biden's campaign for M...,1667528214.0,I couldn't find any live streams and didn't re...,https://www.reddit.com/r/sandiego/comments/ylm...,None
4,Adding to this secret spot posted earlier😍,1667526478.0,,https://v.redd.it/5ajpgv1saux91,Video
...,...,...,...,...,...
95,Turo Car Rental,1667323021.0,I gave up my car earlier this year because I w...,https://www.reddit.com/r/sandiego/comments/yje...,None
96,One of my favorite places to smoke and decompr...,1667322789.0,,https://www.reddit.com/gallery/yjekc4,Photo gallery
97,Any clubs that play non-frat party music?? Any...,1667322566.0,Getting tired of the shitty dj music and frat ...,https://www.reddit.com/r/sandiego/comments/yje...,News
98,Has anyone’s water bill been cut in half?,1667319560.0,So my city of San Diego water bill has consist...,https://www.reddit.com/r/sandiego/comments/yjd...,None


In [9]:
data_df["created_utc"] = pd.to_datetime(data_df["created_utc"], utc=True, unit='s')
data_df

,title,created_utc,selftext,url,link_flair_text
0,4240 La Jolla Village Marriott is swarming wit...,2022-11-04 04:00:10+00:00,,https://i.redd.it/onsyzioxfwx91.jpg,Photo
1,Air Force One with POTUS Landing at MCAS Miram...,2022-11-04 03:19:33+00:00,,https://v.redd.it/aehmjo73rux91,Video
2,How bad are layoffs at your company?,2022-11-04 02:23:00+00:00,,https://www.reddit.com/r/sandiego/comments/ylm...,None
3,Any photos or videos of Biden's campaign for M...,2022-11-04 02:16:54+00:00,I couldn't find any live streams and didn't re...,https://www.reddit.com/r/sandiego/comments/ylm...,None
4,Adding to this secret spot posted earlier😍,2022-11-04 01:47:58+00:00,,https://v.redd.it/5ajpgv1saux91,Video
...,...,...,...,...,...
95,Turo Car Rental,2022-11-01 17:17:01+00:00,I gave up my car earlier this year because I w...,https://www.reddit.com/r/sandiego/comments/yje...,None
96,One of my favorite places to smoke and decompr...,2022-11-01 17:13:09+00:00,,https://www.reddit.com/gallery/yjekc4,Photo gallery
97,Any clubs that play non-frat party music?? Any...,2022-11-01 17:09:26+00:00,Getting tired of the shitty dj music and frat ...,https://www.reddit.com/r/sandiego/comments/yje...,News
98,Has anyone’s water bill been cut in half?,2022-11-01 16:19:20+00:00,So my city of San Diego water bill has consist...,https://www.reddit.com/r/sandiego/comments/yjd...,None


In [10]:
reddit_res['data']['children'][0]['data'].keys()

dict_keys(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext', 'subreddit_name_prefixed', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'upvote_ratio', 'author_flair_background_color', 'ups', 'total_awards_received', 'media_embed', 'thumbnail_width', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'score', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'thumbnail', 'edited', 'author_flair_css_class', 'author_flair_richtext', 'gildings', 'post_hint', 'content_categories', 'is_self', 'subreddit_type', 'created', 'link_flair_type', 'wls', 'removed_by_category', 'banned_by', 'author_flair_type', 'domain', 'allow_live_comments', 'selftext_html',

In [11]:
reddit_res['data'].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])

In [12]:
data_df.to_csv("reddit_cop_example.csv", index=False)

In [13]:
import json
with open('reddit_cop_data.json', 'w') as f:
    json.dump(reddit_res['data'], f)